In [7]:
%pip install lancedb
%pip install pandas
%pip install pyarrow
%pip install ipywidgets
%pip install jupyter
%pip install sentence-transformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 16.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 17.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━

In [13]:
import lancedb
import pandas as pd
import pyarrow as pa
import numpy as np

uri = "runtime_assets/vecdb"
db = lancedb.connect(uri)

In [14]:
db.table_names()

['columns',
 'idm_health',
 'idm_health10',
 'idm_health100',
 'idm_health2',
 'idm_health3',
 'tables']

In [15]:
tbl = db.open_table("idm_health100")
tbl.count_rows()

5418

In [16]:
tbl.search(None).select(["filename"]).to_pandas().values

array([["# Table: 'AbcClassification' (Abc Classification): The classification of inventory, after ABC analysis, into three basic groups for the purpose of stock control and planning. The 3 basic categories a"],
       ["# Table: 'AbuseType' (Abuse Type): When used as a legal term in healthcare, it normally refers to actions that do not involve intentional misrepresentations in billing but which, nevert"],
       ["# Table: 'AccountStatus' (Account Status): The status of the associated account.Ex:- Current- Past due- 30 days past due- 60 days past due- 90 days past due- In collection- Delinquent"],
       ["# Table: 'AccountingBasis' (Accounting Basis): The accounting basis in which transactions that change a company's financial statements are recorded in the periods in which the events occur, rather "],
       ["# Table: 'AccountingFramework' (Accounting Framework): There are many different frameworks of accounting standards in use internationally. Each of these frameworks consists of

In [17]:
tbl.search(None).select(["content"]).to_pandas().values

array([["# Table: 'AbcClassification' (Abc Classification): The classification of inventory, after ABC analysis, into three basic groups for the purpose of stock control and planning. The 3 basic categories a\n# Columns:\n   - AbcClassificationId (integer): The unique identifier of an ABC Classification.\n   - AbcClassificationName (string): The name of an ABC Classification.\n   - AbcClassificationDescription (string): The description of an ABC Classification."],
       ["# Table: 'AbuseType' (Abuse Type): When used as a legal term in healthcare, it normally refers to actions that do not involve intentional misrepresentations in billing but which, nevert\n# Columns:\n   - AbuseTypeId (integer): The unique identifier of a type of abuse.\n   - AbuseTypeName (string): The name of a type of abuse.\n   - AbuseTypeDescription (string): The description of a type of abuse."],
       ["# Table: 'AccountStatus' (Account Status): The status of the associated account.Ex:- Current- Past due- 30 da

In [29]:
# generate an embedding request to ollama  on the sentance 
sentence = "where can I find data about allergy reactions of patients"

import requests
import json

def get_embedding(text):
    response = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": "nomic-embed-text", "prompt": text}
    )
    return response.json()["embedding"]

question_embedding = get_embedding(sentence)
# normalize the embedding to unit length
question_embedding = question_embedding / np.linalg.norm(question_embedding)


In [18]:
from sentence_transformers import SentenceTransformer
# generate an embedding request to ollama  on the sentance 
sentence = "which tables show data about blood tests"
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L12-v2")
# 2. Generate embeddings for a list of sentences
sentence_embeddings = model.encode(sentence)
# 3. Normalize the embeddings to unit length
sentence_embeddings = sentence_embeddings / np.linalg.norm(sentence_embeddings)


In [19]:
df = tbl.search(sentence_embeddings, query_type="vector") \
    .select(["content"]) \
    .limit(10).to_pandas()
df.sort_values(by="content").values


array([["# Table: 'HealthPlanServiceProviderFacilityTest' (Health Plan Service Provider Facility Test): A Test(s) that are offered or available to patients/members under the Health Plan and provided by the Service Provider at the associated Facility effe\n# Columns:\n   - HealthPlanId (long): The unique identifier of a Health Plan.  child reference to HealthPlan.HealthPlanId.\n   - ServiceProviderId (integer): The unique identifier of a Service Provider.  child reference to ServiceProvider.ServiceProviderId.\n   - HealthcareFacilityId (integer): The unique identifier of a Service Provider Facility.  child reference to HealthcareFacility.HealthcareFacilityId.\n   - MedicalTestTypeId (long): The unique identifier of a Medical Test Type.  child reference to MedicalTestType.MedicalTestTypeId.\n   - PeriodStartDate (date): The start period date that the Test is offered by the Service Provider at the associated Facility under the Health Plan.\n   - PeriodEndDate (date): The end period date t

In [20]:
df.sort_values(by="content").values

array([["# Table: 'HealthPlanServiceProviderFacilityTest' (Health Plan Service Provider Facility Test): A Test(s) that are offered or available to patients/members under the Health Plan and provided by the Service Provider at the associated Facility effe\n# Columns:\n   - HealthPlanId (long): The unique identifier of a Health Plan.  child reference to HealthPlan.HealthPlanId.\n   - ServiceProviderId (integer): The unique identifier of a Service Provider.  child reference to ServiceProvider.ServiceProviderId.\n   - HealthcareFacilityId (integer): The unique identifier of a Service Provider Facility.  child reference to HealthcareFacility.HealthcareFacilityId.\n   - MedicalTestTypeId (long): The unique identifier of a Medical Test Type.  child reference to MedicalTestType.MedicalTestTypeId.\n   - PeriodStartDate (date): The start period date that the Test is offered by the Service Provider at the associated Facility under the Health Plan.\n   - PeriodEndDate (date): The end period date t

In [21]:
rel_tables = "\n".join(df['content'].values)

In [22]:
prompt = f"""
You are an AI assistant that answers questions about database schemas and tables.
Here are the relevant tables and columns:
{rel_tables}

Please answer the following question. First explain the purpose of the tables relevant to the question, and if needed provide a query to retrieve the data.
Question:{sentence}
"""

print(prompt)


You are an AI assistant that answers questions about database schemas and tables.
Here are the relevant tables and columns:
# Table: 'MedicalTestType' (Medical Test Type): A type of medical test.Ex:Blood Test- Sodium- Potassium - Serum Iron (TSI)- Ferritin- pH- Oxygen Saturation- Total Protein- Total Bilirubin
# Columns:
   - MedicalTestTypeId (long): The unique identifier of a Medical Test Type.  child reference to HealthcareProduct.ProductId.
   - MedicalTestTypeName (string): The name of a Medical Test Type.
   - MedicalTestTypeDescription (string): The description of a Medical Test Type.
   - MedicalTestCategoryId (integer): The unique identifier of a Medical Test Category.  child reference to MedicalTestCategory.MedicalTestCategoryId.
   - CptCode (string): The unique identifier of a CPT Code.  child reference to CurrentProceduralTerminologyCode.CptCode.
   - HcpcsCode (string): The unique identifier of a HCPCS Code.  child reference to HealthcareCommonProcedureCodingSystemCode.H

In [23]:
import requests
# send request to ollama
response = requests.post(
    "http://localhost:11434/api/generate",
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "options": {
            "temperature": 0.1,
        },
        "stream": False
    }
)

# extract the SQL query from response
sql_query = response.json()["response"]
print("Generated SQL query:")
print(sql_query)

Generated SQL query:
Based on the provided table definitions, I can identify the relevant tables that contain data about blood tests:

1. **MedicalTestResultType**: This table contains information about different types of medical test results, including blood tests. The columns in this table include:
	* MedicalTestResultTypeId (unique identifier)
	* MedicalConditionId (foreign key referencing a medical condition)
	* UnitOfMeasureId (foreign key referencing a unit of measure)

2. **UnitOfMeasure**: This table contains information about different units of measurement, including those used for blood tests. The columns in this table include:
	* UnitOfMeasureId (unique identifier)
	* UnitName (e.g., "mg/dL" or "g/L")

3. **MedicalTestResult**: This table contains data about individual medical test results, including blood tests. The columns in this table include:
	* MedicalTestResultId (unique identifier)
	* TestResultMinimumValue
	* TestResultMaximumValue
	* UnitOfMeasureId (foreign key re